# Prepares data to go into SHETRAN and HIPIMS

## Notes
- This code relies on the following code
    1. `read_met` - reads in 5-minute gridded (composite and processed C-band weather radar from the Met Office ftp server.
    2. `read_ea` - reads in 15-minute rain gauge data from the EA website API.
    3. `read_cs` - reads in varying resolution rain gauge data from NGIF and UO APIand accumulates up to 15-minute resolution data.
    4. `read_cs_radar` - reads in Urban Observatory X-band radar code, processes it and accumulates it up to 15 minutes.
    5. `intense_qc` - quality controls all gauge data using intense-qc code (Note: this is designed for hourly rain gauge data). 
- It also requires the bounding box shapefile for HIPIMS and the Tyne mask file for SHETRAN (these are in the `static.zip` sent on teams).

## What does this code do?

### Reads in data
1. Read in radar data (if exists)
    - Met Office radar data
    - Urban Observatory radar data
2. Read in quality controlled rain gauge data (if exists)
    - Environment Agency rain gauges
    - Urban Observatory rain gauges
    - Citizen Science rain gauges
    - National Green Infrastructure Facility rain gauges
3. Identifies best data to use
    - Uses data priority for HIPIMS
    - Uses data priority for SHETRAN
4. Grids/merges data
    - Combines different data types to provide gridded 15-minute rainfall estimates (1km resolution)
4. Saves data in correct format for models
    - Clips and saves data in correct format for HIPIMS
    - Clips and saves data in correct format for SHETRAN
    
### Prioritises data for input into HIPIMS model
1. High resolution (250m) X-band radar data, merged with rain gauges, which is conditional on Met Office C-band radar data.
2. High resolution (250m) X-band radar data, conditional on Met Office C-band radar data.
3. High resolution (250m) X-band radar data, merged with rain gauges.
4. Merged Met Office radar data and quality controlled rain gauge data.
5. Met Office radar data.
6. Gridded quality controlled rain gauge data (IF Environment Agency rain gauge data exists).

NOTE: 1. and 2. are missing as cannot test this whilst the Urban Observatory radar is down for maintenence and API is not working.

### Prioritises data for input into SHETRAN model
1. Merged Met Office radar data and quality controlled rain gauge data.
2. Met Office radar data.
3. Gridded quality controlled rain gauge data (IF Environment Agency rain gauge data exists). 


## File structure

### Data inputs format 
- `\inputs` - root inputs data folder

### Data outputs format
- `\outputs` - root outputs data folder
    - `\SHETRAN` - SHETRAN rainfall input data, which should be in the correct format to run the model if running at 15 minute resolution (Note: We somehow need to add start date and data resolution into the library shetran set-up file I think).
    - `\HIPIMS` - HIPIMS rainfall input data, which should be in the correct format to run the model at 15 minute resolution

## Data minimum requirements

### Minimum data requirement to run model
Either Met Office C-band radar data, or Environment Agency rain gauge data.


- SHETRAN 1 and 2
    1. Requirements:
        - `\input\MET` exists and contains the files `arrays.npy`, `timestamp.csv`, `coords_x.csv` and `coords_y.csv`
    2. Additional information if available:
        - `\input\EA\15min` exists and contains any `.csv` files
        - `\input\UO\15min` exists and contains any `.csv` files
        - `\input\CS\15min` exists and contains any `.csv` files
        - `\input\NGIF\15min` exists and contains any `.csv` files


- SHETRAN 3
    1. Requirements:
        - `\input\EA\15min` exists and contains more than one `.csv` file
    2. Additional information if available:
        - `\input\UO\15min` exists and contains any `.csv` files
        - `\input\CS\15min` exists and contains any `.csv` files
        - `\input\NGIF\15min` exists and contains any `.csv` files


- HIPIMS 1
    1. Requirements:
        - `\input\UO\radar\15min` exists and contains the files `arrays.npy`, `timestamp.csv`, `coords_x.csv` and `coords_y.csv`
    - `\input\MET` exists and contains the files `arrays.npy`, `timestamp.csv`, `coords_x.csv` and `coords_y.csv`
    2. Additional information if available:
        - `\input\EA\15min` exists and contains any `.csv` files
        - `\input\UO\15min` exists and contains any `.csv` files
        - `\input\CS\15min` exists and contains any `.csv` files
        - `\input\NGIF\15min` exists and contains any `.csv` files


- HIPIMS 2
    1. Requirements:
        - `\input\UO\radar\15min` exists and contains the files `arrays.npy`, `timestamp.csv`, `coords_x.csv` and `coords_y.csv`
        - `\input\MET` exists and contains the files `arrays.npy`, `timestamp.csv`, `coords_x.csv` and `coords_y.csv`
    2. Additional information if available:
        - `\input\EA\15min` exists and contains any `.csv` files
        - `\input\UO\15min` exists and contains any `.csv` files
        - `\input\CS\15min` exists and contains any `.csv` files
        - `\input\NGIF\15min` exists and contains any `.csv` files

In [1]:
# Import relevent packages
from os.path import join, exists, split
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import wradlib.ipol as ipol
import gstools as gs

In [2]:
# file paths to change
input_path = r"C:\Users\Amy\OneDrive - Newcastle University (1)\Documents\PYRAMID\data\realtime"

# output data paths
output_path = input_path

# Bounding box for data 
e_l, n_l, e_u, n_u = [355000, 534000, 440000, 609000]
bbox = [e_l, e_u, n_l, n_u]
res = 1000

output_shetran_path = join(output_path, "SHETRAN")
if not exists(output_shetran_path):
    os.mkdir(output_shetran_path)

output_hipims_path = join(output_path, "HIPIMS")
if not exists(output_hipims_path):
    os.mkdir(output_hipims_path)

# path to files needed to clip data (e.g. bounding box for HIPIMS, mask file for SHETRAN)
static_path = join(input_path, "static")

# set up a function that reads the header lines from a SHETRAN mask:
def read_ascii_header(file_path: str):

    header_dict = {}
    line=[]

    with open(file_path, 'r') as f:
        while True:
            line = f.readline()
            line = line.split()
            if len(line)>2: break
            header_dict[line[0]] = float(line[1])

    return header_dict
    
# path to SHETRAN mask (needs changing)
shetran_mask_path = join(static_path, "Tyne_at_Newcastle_Mask.asc")

In [3]:
# helper function for reading in rain gauge data
def read_gauges(source):
    
    gauges = []
    
    root_path = join(input_path, source)
    folder_path = join(root_path, "15min")
    
    if exists(folder_path):

        for f in os.listdir(folder_path):
            if f.endswith(".csv"):
                
                data = pd.read_csv(join(folder_path, f), index_col=0)
                data.index = pd.to_datetime(data.index, utc=True)
                data.columns = [source + "_" + f.split(".")[0]]
                gauges.append(data)
                
    return gauges

In [4]:
met_path = join(join(input_path, "MET"), "15min")
uo_path = join(input_path, "UO")
uo_radar_path = join(join(uo_path, "radar"), "15min")

shetran_method = np.nan
hipims_method = np.nan

radar_files = ['arrays.npy', 'coords_x.csv', 'coords_y.csv', 'timestamp.csv']
if exists(met_path):
    if all([f in os.listdir(met_path) for f in radar_files]):
        
        # try and read in Met Office weather radar data 
        met_radar = {}
        for f in radar_files:
            if f.endswith(".csv"):
                data = pd.read_csv(join(met_path, f)).iloc[:, 0]
            else:
                data = np.load(join(met_path, f))

            met_radar[f.split(".")[0]] = data
        
        # try and read in rain gauge data
        gauges = []
        for source in ["EA", "CS", "UO", "NGIF"]:
            gauges.extend(read_gauges(source))

        if len(gauges) > 0:
            gauges = pd.concat(gauges, 1)
            
            # SHETRAN INPUT 1: merge radar and gauges together
            shetran_method = 1
            
        else:
            
            # SHETRAN INPUT 2: just radar data
            shetran_method = 2
else:
    
    gauges = []
    for source in ["EA", "CS", "UO", "NGIF"]:
        gauges.extend(read_gauges(source))
    if len(gauges) > 0:
        gauges = pd.concat(gauges, 1)
        
        # check Environment Agency gauge data available
        if sum([col.split("_")[0] == "EA" for col in gauges.columns]) > 1: 
            
            # SHETRAN INPUT 3: gridded rain gauge data
            shetran_method = 3

if exists(uo_radar_path):
    
    if all([f in os.listdir(uo_radar_path) for f in radar_files]):
        
        # try and read in Urban Observatory weather radar data 
        uo_radar = {}
        for f in radar_files:
            if f.endswith(".csv"):
                data = pd.read_csv(join(uo_radar_path, f)).iloc[:, 0]
            else:
                data = np.load(join(uo_radar_path, f))

            uo_radar[f.split(".")[0]] = data
            
        if uo_radar["arrays"].shape[0] > 0:
            
            hipims_method = shetran_method
            
            # HIPIMS INPUT 1, 2 or 3: High-resolution X-band radar data 

C:\Users\Amy\AppData\Local\Temp\ipykernel_16696\829378489.py:28: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  gauges = pd.concat(gauges, 1)


In [5]:
if not np.isnan(hipims_method):
    
    if hipims_method == 1:
        
        # HIPIMS INPUT 1: High-resolution X-band radar data, merged with rain gauges, which is conditional on Met Office C-band radar data.
        
        """
        # NOTE: Code not written as high-resolution X-band radar data 
                was down for maintenence/broken for the duration of 
                the PYRAMID project, but this is where the code would
                go for high resolution (250m) X-band radar data, merged 
                with rain gauges, which is conditional on Met Office 
                C-band radar data.
                
                If code is included, output should be a dictionary 
                named high_res_rainfall, with attributes mirroring 
                full_rainfall.
        """
        
    elif hipims_method == 2:
        
        # HIPIMS INPUT 2: High-resolution X-band radar data, conditional on Met Office C-band radar data
        
        """
        # NOTE: Code not written as high-resolution X-band radar data 
                was down for maintenence/broken for the duration of 
                the PYRAMID project, but this is where the code would
                go for high resolution (250m) X-band radar data, 
                conditional on Met Office C-band radar data.
                                
                If code is included, output should be a dictionary 
                named high_res_rainfall, with attributes mirroring 
                full_rainfall.
        """
        
    elif hipims_method == 3:
        
        # HIPIMS INPUT 3: High-resolution X-band radar data, merged with rain gauges.
        
        """
        # NOTE: Code not written as high-resolution X-band radar data 
                was down for maintenence/broken for the duration of 
                the PYRAMID project, but this is where the code would
                go for high resolution (250m) X-band radar data, 
                merged with rain gauges.
                                
                If code is included, output should be a dictionary 
                named high_res_rainfall, with attributes mirroring 
                full_rainfall.
        """

In [ ]:
# set up new coordinates for SHETRAN
mask = np.loadtxt(shetran_mask_path, skiprows=6)

# get mask meta info
meta = read_ascii_header(shetran_mask_path)

new_x = np.arange(meta["xllcorner"], 
                  meta["xllcorner"] + meta["ncols"] * meta["cellsize"],
                  meta["cellsize"])

new_y = np.arange(meta["yllcorner"], 
                  meta["yllcorner"] + meta["nrows"] * meta["cellsize"],
                  meta["cellsize"])

if not np.isnan(shetran_method):
    if shetran_method == 1:

        # SHETRAN INPUT 1: Merged Met Office radar data and quality controlled rain gauge data.
        full_rainfall = {}
        for attr in ["timestamp", "coords_x", "coords_y"]:
            full_rainfall[attr] = met_radar[attr]

        full_rainfall["arrays"] = np.full(met_radar["arrays"].shape, np.nan)

        g_x = np.array([col.split("_")[-2] for col in gauges.columns]).astype(float)
        g_y = np.array([col.split("_")[-1] for col in gauges.columns]).astype(float)

        # source coordinates
        src = np.vstack([g_x, g_y]).transpose()

        # target coordinates
        xtrg = full_rainfall["coords_x"]
        ytrg = np.flip(full_rainfall["coords_y"]) # check this is correct way round

        trg = np.meshgrid(xtrg, ytrg)
        trg = np.vstack((trg[0].ravel(), trg[1].ravel())).T

        for t in range(met_radar["arrays"].shape[0]):
            
            rad = met_radar["arrays"][t]
            
            try:
                
                gau = np.array(gauges.iloc[t, :])

                rad_x = [np.argmin(abs(x - met_radar["coords_x"])) for x in g_x]
                rad_y = [np.argmin(abs(y - np.flip(met_radar["coords_y"]))) for y in g_y]

                rad_at_g = np.array(rad[rad_y, rad_x])

                try:
                    # interpolation objects (Kriging with External Drift)
                    if not np.isnan(np.nansum(rad_at_g)):
                        ked = ipol.ExternalDriftKriging(src, trg, src_drift=rad_at_g, trg_drift=rad.flatten()) 
                        # should investigate further variogram structure for data
                        # default set to cov='1.0 Exp(10000.)'
                    else:
                        ked = ipol.ExternalDriftKriging(src, trg)

                    full_rainfall["arrays"][t] = ked(gau).reshape(rad.shape) 
                
                except:
                    full_rainfall["arrays"][t] = rad
            
            except:
                full_rainfall["arrays"][t] = rad
                    
    elif shetran_method == 2:

        # SHETRAN INPUT 2: Met Office radar data.
        full_rainfall = met_radar

    elif shetran_method == 3:

        # SHETRAN INPUT 3: Gridded quality controlled rain gauge data (IF Environment Agency rain gauge data exists). 
        full_rainfall = {}
        full_rainfall["timestamp"] = gauges.index

        g_x = np.array([col.split("_")[-2] for col in gauges.columns]).astype(float)
        g_y = np.array([col.split("_")[-1] for col in gauges.columns]).astype(float)

        full_rainfall["coords_x"] = new_x
        full_rainfall["coords_y"] = new_y

        # source coordinates
        src = np.vstack([g_x, g_y]).transpose()

        # target coordinates
        xtrg = full_rainfall["coords_x"]
        ytrg = full_rainfall["coords_y"]
        trg = np.meshgrid(xtrg, ytrg)
        trg = np.vstack((trg[0].ravel(), trg[1].ravel())).T

        # interpolation objects (Ordinary Kriging)
        ok = ipol.OrdinaryKriging(src, trg, remove_missing=True)
        ok1 = ipol.OrdinaryKriging(src[0:-1, :], trg, remove_missing=True)

        gridded = np.full((len(gauges.index), len(ys), len(xs)), np.nan) # check dimensions correct way round

        for t in range(len(gauges.index)):

            vals = gauges.iloc[t].values
            tot = np.nansum(vals)

            if not np.isnan(tot):
                if tot == 0:
                    gridded[t] = 0
                else:
                    try:
                        gridded[t] = np.flip(ok(vals).astype(float).reshape((len(ys), len(xs))), axis=0)
                    except:
                        pass    

        full_rainfall["arrays"] = gridded
    
else:
    
    # INSUFFICIENT DATA FOR THE CODE TO RUN
    print("Insufficient data for the code to run. Stop the workflow.")

In [93]:
# clip to SHETRAN domain and save

output_shetran_full = np.full(
    (full_rainfall["arrays"].shape[0], 
     new_y.shape[0], 
     new_x.shape[0]), np.nan)

xs = full_rainfall["coords_x"].values
ys = np.flip(full_rainfall["coords_y"].values)

new_x_cond = np.array([min(abs(x - xs)) < 500 for x in new_x])
new_y_cond = np.array([min(abs(y - ys)) < 500 for y in new_y])

old_x_cond = np.array([min(abs(x - new_x)) < 500 for x in xs])
old_y_cond = np.array([min(abs(y - new_y)) < 500 for y in ys])

if not all(old_y_cond):
    input1 = full_rainfall["arrays"][:, old_y_cond, :]
else:
    input1 = full_rainfall["arrays"]
    
if not all(old_x_cond):
    input2 = input1[:, :, old_x_cond]
else:
    input2 = input1

if all(new_x_cond) and all(new_y_cond):
    output_shetran_full = input2
elif all(new_x_cond):
    output_shetran_full[:, new_y_cond, :] = input2
else:
    output_shetran_full[:, new_y_cond, :][:, :, new_x_cond] = input2

output_shetran = pd.DataFrame(output_shetran_full[:, mask == 0])
output_shetran.to_csv(join(output_shetran_path, "Tyne_at_Newcastle_Precip.csv"), index=False)

In [110]:
# clip data to HIPIMS domain and save

# path to HIPIMS bbox file
hipims_bbox_path = join(static_path, "boundbox.shp")
bbox = gpd.read_file(hipims_bbox_path)

# reproject bouninging box for data
bbox['geometry'] = bbox['geometry'].to_crs(epsg=27700)
min_x, min_y, max_x, max_y = bbox.bounds.loc[0]

if not np.isnan(hipims_method):
    
    # high-resolution data 
    try:
        """
        Note: This does not work as high_res_rainfall dictionary 
        not defined as code does not existdo to maintence issues 
        of high resolution X-band radar data at Urban Observatory.
        """
        
        xs = high_res_rainfall["coords_x"]
        ys = high_res_rainfall["coords_y"]
        arrs = high_res_rainfall["arrays"]
        
    except:
        print("High resolution rainfall does not exist.")
        xs = full_rainfall["coords_x"]
        ys = full_rainfall["coords_y"]
        arrs = full_rainfall["arrays"]
    
else:
    
    # no high-resolution data
    xs = full_rainfall["coords_x"]
    ys = full_rainfall["coords_y"]
    arrs = full_rainfall["arrays"]
    
    
x_cond = (xs >= min_x - 500) & (xs <= max_x + 500)
y_cond = (ys >= min_y - 500) & (ys <= max_y + 500)
clip_x = arrs[:, :, x_cond]
clipped_rainfall = clip_x[:, y_cond[::-1], :]

# Reformat to dataframe for input file
data = clipped_rainfall.reshape(clipped_rainfall.shape[0], clipped_rainfall.shape[1] * clipped_rainfall.shape[2])
clipped_rainfall_hipims = pd.DataFrame(data, index=full_rainfall["timestamp"])

# Save file
clipped_rainfall_hipims.to_csv(join(output_hipims_path, "rain_source.txt"))